In [151]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from sklearn.model_selection import KFold

fires = pd.read_csv("fire_data_processed_after_pca.csv")
X = fires
cut = 1
y = np.exp(X.loc[:,"area"]) - 1 - cut
X = X.drop(["area", "area_bool"], axis=1)
feat_drop = ['apr_bool', '66', '73', '65', '74', '64', '75', '95', '63', '99', '84', '55', '13', 'aug_bool', '76', '43', '44', '85', 'rain', '94', 'oct_bool', '45', '33', 'nov_bool', 'feb_bool', '23',
 'wed', 'mar_bool', 'fri', 'jun_bool', '14', 'tue', '24', '88', '46', 'sat']

features_added = [["22","sat"], ["wed","pc5"], ["tue","pc2"], ["94", "pc6"], ['sep_bool','pc4']]
  

def add_feature(df, col1, col2):
    df = df.copy()
    new_name = col1 + " * " + col2
    map_names = {"col_name":new_name}
    return (df.assign(col_name = df.loc[:,col1] * df.loc[:,col2])).rename(map_names, axis=1)

def l2(theta):
    return theta@theta.T

def exp_loss(weights, X, y):
    return sum(np.exp(-y*(X@weights.T)))

def hinge_loss(weights, X, y):
    loss = (-(weights@X.T)+1) * y
    loss[loss<0] = 0
    return np.sum(loss)

def objective(X, y, weights, lambd =0.5, loss = exp_loss, regularizer=l2):
    return loss(weights, X, y) + lambd*regularizer(weights)

for feat in features_added:
    X = add_feature(X, feat[0], feat[1])
X = X.drop(feat_drop, axis=1)

regs = np.linspace(0.001, 3, 5)
cutoffs = np.linpsace(0.001, 2, 5)
rskf = KFold(n_splits=4, shuffle=True,random_state=3558)
dic = {}
for r in regs:
    for c in cutoffs:
        success = 0
        missclass = 0
        cum_par = np.zeros(1, X.shape[1])
        for train_index, test_index in rskf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            x0 = np.random.rand(1, X_train.shape[1])
            res = minimize(lambda weights: objective(X_train, y_train, weights, loss=hinge_loss), x0 = x0, method = 'Nelder-Mead')
            if res.success:
                success += 1
                pred = ((1-(X_test @ res.x)) < 0)
                missclass += sum(pred* y_test <0)
                cum_par += res.x
        temp_str = str(r) + str(c)
        dic[temp_str] = {"missclassified": missclass/sucess, "parameters":cum_par/success, "successfull iterations":success}

 final_simplex: (array([[0.00322315, 0.27049142, 0.81317686, ..., 0.77101526, 0.52187726,
        0.93075913],
       [0.00322298, 0.27049051, 0.81318341, ..., 0.77101332, 0.52187832,
        0.93075055],
       [0.00322305, 0.27048785, 0.81318714, ..., 0.77100681, 0.52187771,
        0.93072858],
       ...,
       [0.003223  , 0.27048769, 0.81318171, ..., 0.77100332, 0.52187649,
        0.93073082],
       [0.00322304, 0.27049215, 0.81317282, ..., 0.77101208, 0.52187752,
        0.93076464],
       [0.00322295, 0.27049102, 0.81317762, ..., 0.77100963, 0.52187797,
        0.93075508]]), array([125.8919428 , 125.8919556 , 125.8919565 , 125.8919597 ,
       125.89196593, 125.89196852, 125.89197003, 125.89197436,
       125.89198091, 125.89198366, 125.89198411, 125.89198428,
       125.89198864, 125.8919887 , 125.89198917, 125.89199658,
       125.89199888, 125.89199893, 125.89200175, 125.89200406,
       125.89200414, 125.89200645, 125.89201083, 125.89201123,
       125.89201132, 125.89

In [166]:
weights = [0.00322315,  0.27049142,  0.81317686,  0.25891973,  0.46838279,
        0.43703944,  0.25834756,  0.61129018,  0.13451186,  0.11451814,
        0.03167031,  0.21261865, -0.46339581, -0.05297954,  0.60704822,
        0.92383392,  0.05269001,  0.32568387,  0.09451929, -0.07845529,
        0.09921569,  0.18505701,  0.69258364,  0.8411936 ,  0.50037871,
        0.41329365,  0.91072145,  0.75318558,  0.04080461,  0.47482101,
        0.22131864,  0.77101526,  0.52187726,  0.93075913]
pred = ((1-(X @ weights)) < 0)

In [170]:
sum(pred* y <0)

181

In [171]:
x0 = np.random.rand(1, X.shape[1])
res = minimize(lambda weights: objective(X, y, weights, loss=hinge_loss), x0 = x0, method = 'Nelder-Mead')

In [197]:
w = [0.00528337,  0.50690796,  0.27307986,  0.26020999,  0.82160399,
          -0.04517859,  0.26659104,  0.96187361,  0.04980793,  0.65898698,
           0.44550089,  0.58260576,  0.59906079,  0.04932569,  1.11683321,
           0.01732238,  0.11639177,  0.19671073,  0.08213164,  0.11132968,
           0.34619528,  0.04484096,  0.03340923,  0.02910226,  0.2251468 ,
           0.34634532,  0.81110492,  0.63381886,  0.09352773,  0.39075299,
           1.09733269,  0.63982484,  1.35402455,  0.33949129]

pred = ((1-(X @ w[1:])) < 0)
sum(pred* y <=0)

509

In [174]:
res.success

False

In [188]:
X = X.drop(["Unnamed: 0"], axis=1)